## Action items
1. spark read delta table from adls, use expression to group by for latestchanges.
2. call write api from adb to synapse sql dedicated pool to store latestchanges.
3. auth to sql pool using keyvault

In [13]:
val accountName = "wplushiramsynapseadlsv2"
val accountKey = ""
val containerName = "wplushiramsynapsefs"
val deltaPath = "synapse/workspaces/wplushiramsynapse/warehouse/lastchanges"
val jdbcConnectionString = ""

In [6]:
val df_delta = spark.read.format("delta").load("abfss://"+containerName+"@"+accountName+".dfs.core.windows.net/"+deltaPath)
df_delta.show()

In [8]:
val latestonly = df_delta.selectExpr("key", "struct(time, newValue, deleted) as otherCols").groupBy("key").agg(max("otherCols").as("latest")).selectExpr("key", "latest.*")
latestonly.show()

https://stackoverflow.com/questions/26041479/query-sql-database-with-scala

In [10]:
Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")
import java.sql.{Connection, DriverManager, ResultSet, Statement}; 

def dt = {
    val connection_string = jdbcConnectionString
    val connection = DriverManager.getConnection(connection_string)
    try {
        val statement = connection.createStatement()
        val results = statement.executeQuery(" if object_id('latestonly') is not null drop table latestonly; select 1")
    } finally {
        connection.close
    } 
}

dt

In [11]:
latestonly.write.option(Constants.SERVER, "wplushiramsynapse.sql.azuresynapse.net").synapsesql("wplussynapsedw.dbo.latestonly", Constants.INTERNAL)

## But he wants it in ADB.
1. https://docs.databricks.com/data/data-sources/azure/synapse-analytics.html#frequently-asked-questions-faq
2. https://github.com/Azure-Samples/azure-sql-db-databricks/blob/main/notebooks/03b-parallel-switch-in-load-into-partitioned-table-single.ipynb